In [35]:
# First ensemble with NSL-KDD
# Parameters

#----------------------------------------------
# 0 for not using it as base learner
# 1 for using it as base learner

use_model_ada = 1 
use_model_dnn = 1 
use_model_mlp = 1 
use_model_lgbm = 1 
use_model_rf = 1 
use_model_svm = 1
use_model_knn = 1 
#----------------------------------------------
# 0 for training the model
# 1 for using the saved version of the model

load_model_ada = 0 
load_model_dnn = 0 
load_model_mlp = 0 
load_model_lgbm = 0 
load_model_rf = 0 
load_model_svm = 0
load_model_knn = 0 
#----------------------------------------------

# load_model_ada = 1
# load_model_dnn = 1 
# load_model_mlp = 1 
# load_model_lgbm = 1 
# load_model_rf = 1 
# load_model_svm = 1
# load_model_knn = 1 
#----------------------------------------------
feature_selection_bit = 0
# feature_selection_bit = 1




In [36]:

# Specify the name of the output text file
if feature_selection_bit == 0:
    output_file_name = "ensemble_base_models_all_features.txt"
    with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)
    with open(output_file_name, "a") as f: print('---- ensemble_base_models_all_features', file = f)

elif feature_selection_bit == 1:
    output_file_name = "ensemble_base_models_feature_selection.txt"
    with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)
    with open(output_file_name, "a") as f: print('----ensemble_base_models_feature_selection--', file = f)


In [37]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:
# importing required libraries
import numpy as np
import pandas as pd
import pickle # saving and loading trained model
from os import path


# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import (StandardScaler, OrdinalEncoder,LabelEncoder, MinMaxScaler, OneHotEncoder)
from sklearn.preprocessing import Normalizer, MaxAbsScaler , RobustScaler, PowerTransformer

# importing library for plotting
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from keras.layers import Dense # importing dense layer

from keras.layers import Input
from keras.models import Model
# representation of model layers
#from keras.utils import plot_model
from sklearn.metrics import confusion_matrix
import shap

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
import time
start_program = time.time()



In [38]:


def confusion_metrics (name_model,predictions,true_labels):

    name = name_model
    pred_label = predictions
    y_test_01 = true_labels 

    with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print(name, file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    # voting_acc_01 = Acc
    # voting_pre_01 = Precision
    # weighed_avg_rec_01 = Recall
    # weighed_avg_f1_01 = F1
    # weighed_avg_bacc_01 = BACC
    # weighed_avg_mcc_01 = MCC
    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)

    return Acc, Precision, Recall, F1, BACC, MCC


In [39]:



# In[3]:


# attach the column names to the dataset
feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]
# KDDTrain+_2.csv & KDDTest+_2.csv are the datafiles without the last column about the difficulty score
# these have already been removed.

train='KDDTrain+.txt'
test='KDDTest+.txt'

df=pd.read_csv(train,names=feature)
df_test=pd.read_csv(test,names=feature)


In [40]:


# shape, this gives the dimensions of the dataset
print('Dimensions of the Training set:',df.shape)
print('Dimensions of the Test set:',df_test.shape)


df.drop(['difficulty'],axis=1,inplace=True)
df_test.drop(['difficulty'],axis=1,inplace=True)



print('Label distribution Training set:')
print(df['label'].value_counts())
print()
print('Label distribution Test set:')
print(df_test['label'].value_counts())



# colums that are categorical and not binary yet: protocol_type (column 2), service (column 3), flag (column 4).
# explore categorical features
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

#see how distributed the feature service is, it is evenly distributed and therefore we need to make dummies for all.
print()
print('Distribution of categories in service:')
print(df['service'].value_counts().sort_values(ascending=False).head())



# Test set
print('Test set:')
for col_name in df_test.columns:
    if df_test[col_name].dtypes == 'object' :
        unique_cat = len(df_test[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))


from sklearn.preprocessing import LabelEncoder,OneHotEncoder
categorical_columns=['protocol_type', 'service', 'flag']
# insert code to get a list of categorical columns into a variable, categorical_columns
categorical_columns=['protocol_type', 'service', 'flag'] 
 # Get the categorical values into a 2D numpy array
df_categorical_values = df[categorical_columns]
testdf_categorical_values = df_test[categorical_columns]
df_categorical_values.head()


# protocol type
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
# service
unique_service=sorted(df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
# flag
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
# put together
dumcols=unique_protocol2 + unique_service2 + unique_flag2
print(dumcols)

#do same for test set
unique_service_test=sorted(df_test.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2




df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)
print(df_categorical_values_enc.head())
# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)



enc = OneHotEncoder()
df_categorical_values_encenc = enc.fit_transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc.toarray(),columns=dumcols)
# test set
testdf_categorical_values_encenc = enc.fit_transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc.toarray(),columns=testdumcols)

df_cat_data.head()


trainservice=df['service'].tolist()
testservice= df_test['service'].tolist()
difference=list(set(trainservice) - set(testservice))
string = 'service_'
difference=[string + x for x in difference]
difference

for col in difference:
    testdf_cat_data[col] = 0

testdf_cat_data.shape

newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=df_test.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)


# take label column
labeldf=newdf['label']
labeldf_test=newdf_test['label']
# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
# put the new label column back
newdf['label'] = newlabeldf
newdf_test['label'] = newlabeldf_test
print(newdf['label'].head())


# Specify your selected features. Note that you'll need to modify this list according to your final processed dataframe
#Uncomment the below lines to use these top 20 features from shap analysis
#selected_features = ["root_shell","service_telnet","num_shells","service_uucp","dst_host_same_src_port_rate"
#                     ,"dst_host_rerror_rate","dst_host_srv_serror_rate","dst_host_srv_count","service_private","logged_in",
#                    "dst_host_serror_rate","serror_rate","srv_serror_rate","flag_S0","diff_srv_rate","dst_host_srv_diff_host_rate","num_file_creations","flag_RSTR"#,"dst_host_same_srv_rate","service_Idap","label"]
                     

# Select those features from your dataframe
#newdf = newdf[selected_features]
#newdf_test = newdf_test[selected_features]

# Now your dataframe only contains your selected features.

# creating a dataframe with multi-class labels (Dos,Probe,R2L,U2R,normal)
multi_data = newdf.copy()
multi_label = pd.DataFrame(multi_data.label)

multi_data_test=newdf_test.copy()
multi_label_test = pd.DataFrame(multi_data_test.label)


# using standard scaler for normalizing
std_scaler = StandardScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = multi_data.select_dtypes(include='number').columns
data = standardization(multi_data,numeric_col)
numeric_col_test = multi_data_test.select_dtypes(include='number').columns
data_test = standardization(multi_data_test,numeric_col_test)

# label encoding (0,1,2,3,4) multi-class labels (Dos,normal,Probe,R2L,U2R)
le2 = preprocessing.LabelEncoder()
le2_test = preprocessing.LabelEncoder()
enc_label = multi_label.apply(le2.fit_transform)
enc_label_test = multi_label_test.apply(le2_test.fit_transform)
multi_data = multi_data.copy()
multi_data_test = multi_data_test.copy()

multi_data['intrusion'] = enc_label
multi_data_test['intrusion'] = enc_label_test

#y_mul = multi_data['intrusion']
multi_data
multi_data_test



multi_data.drop(labels= [ 'label'], axis=1, inplace=True)
multi_data
multi_data_test.drop(labels= [ 'label'], axis=1, inplace=True)
multi_data_test


y_train_multi= multi_data[['intrusion']]
X_train_multi= multi_data.drop(labels=['intrusion'], axis=1)

print('X_train has shape:',X_train_multi.shape,'\ny_train has shape:',y_train_multi.shape)

y_test_multi= multi_data_test[['intrusion']]
X_test_multi= multi_data_test.drop(labels=['intrusion'], axis=1)

print('X_test has shape:',X_test_multi.shape,'\ny_test has shape:',y_test_multi.shape)


from collections import Counter

label_counts = Counter(y_train_multi['intrusion'])
print(label_counts)


from sklearn.preprocessing import LabelBinarizer

y_train_multi = LabelBinarizer().fit_transform(y_train_multi)

y_test_multi = LabelBinarizer().fit_transform(y_test_multi)


Y_train=y_train_multi.copy()
X_train=X_train_multi.copy()

Y_test=y_test_multi.copy()
X_test=X_test_multi.copy()




Dimensions of the Training set: (125973, 43)
Dimensions of the Test set: (22544, 43)
Label distribution Training set:
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: label, dtype: int64

Label distribution Test set:
normal             9711
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess  

X_train has shape: (125973, 122) 
y_train has shape: (125973, 1)
X_test has shape: (22544, 122) 
y_test has shape: (22544, 1)
Counter({0: 67343, 1: 45927, 2: 11656, 3: 995, 4: 52})


In [41]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification

# Assuming you have features X and labels Y
# X, Y = make_classification()

ros = RandomOverSampler(sampling_strategy='minority', random_state=100)

X_train, Y_train = ros.fit_resample(X_train, Y_train)


# In[33]:


print(Y_test)


# In[34]:


X_train.values


# In[35]:



[[0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 ...
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]]


array([[-1.10249223e-01, -7.67859947e-03, -4.91864438e-03, ...,
        -1.97262160e-02,  8.25150071e-01, -4.64315895e-02],
       [-1.10249223e-01, -7.73736981e-03, -4.91864438e-03, ...,
        -1.97262160e-02,  8.25150071e-01, -4.64315895e-02],
       [-1.10249223e-01, -7.76224074e-03, -4.91864438e-03, ...,
        -1.97262160e-02, -1.21190076e+00, -4.64315895e-02],
       ...,
       [-9.29714678e-02, -7.36430591e-03, -3.87394518e-03, ...,
        -1.97262160e-02,  8.25150071e-01, -4.64315895e-02],
       [-8.68282658e-02, -7.36430591e-03, -3.87568593e-03, ...,
        -1.97262160e-02,  8.25150071e-01, -4.64315895e-02],
       [ 1.61587463e-01, -7.46804833e-03,  1.06953862e-03, ...,
        -1.97262160e-02,  8.25150071e-01, -4.64315895e-02]])

In [42]:
single_class_train = np.argmax(y_train_multi, axis=1)
single_class_test = np.argmax(y_test_multi, axis=1)


df1 = X_train_multi.assign(Label = single_class_train)
df2 =  X_test_multi.assign(Label = single_class_test)

frames = [df1,  df2]

df = pd.concat(frames,ignore_index=True)


if feature_selection_bit == 1:
    feature_selection = [
                        'dst_host_same_srv_rate',
                        'dst_host_srv_count',
                        'dst_host_same_src_port_rate',
                        'logged_in',
                        'dst_host_serror_rate',
                        'count',
                        'srv_count',
                        'dst_host_rerror_rate',
                        'Label'
                        ]

    df_og = df
    df = df[feature_selection]





In [43]:

y = df.pop('Label')
X = df

y1, y2 = pd.factorize(y)

y_0 = pd.DataFrame(y1)
y_1 = pd.DataFrame(y1)
y_2 = pd.DataFrame(y1)
y_3 = pd.DataFrame(y1)
y_4 = pd.DataFrame(y1)


# y_0 = y_0.replace(0, 0)
# y_0 = y_0.replace(1, 1)
y_0 = y_0.replace(2, 1)
y_0 = y_0.replace(3, 1)
y_0 = y_0.replace(4, 1)


y_1 = y_1.replace(1, 999)
y_1 = y_1.replace(0, 1)
# y_1 = y_1.replace(1, 0)
y_1 = y_1.replace(2, 1)
y_1 = y_1.replace(3, 1)
y_1 = y_1.replace(4, 1)
y_1 = y_1.replace(999, 1)


y_2 = y_2.replace(0, 1)
y_2 = y_2.replace(1, 1)
y_2 = y_2.replace(2, 0)
y_2 = y_2.replace(3, 1)
y_2 = y_2.replace(4, 1)


y_3 = y_3.replace(0, 1)
# y_3 = y_3.replace(1, 1)
y_3 = y_3.replace(2, 1)
y_3 = y_3.replace(3, 0)
y_3 = y_3.replace(4, 1)


y_4 = y_4.replace(0, 1)
# y_4 = y_4.replace(1, 1)
y_4 = y_4.replace(2, 1)
y_4 = y_4.replace(3, 1)
y_4 = y_4.replace(4, 0)



df = df.assign(Label = y)

In [44]:
#Divide the dataset between level 00 and level 01
import sklearn
from sklearn.model_selection import train_test_split
split = 0.7 # 0.5

# X_00,X_01, y_00, y_01 = sklearn.model_selection.train_test_split(X, y, train_size=split)
X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=split)

In [45]:
from collections import Counter

label_counts2 = Counter(y)
print(label_counts2)


Counter({0: 77054, 1: 53387, 2: 14077, 3: 3880, 4: 119})


In [46]:
#Base learner Split
# split = 0.7

# X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_00, y_00, train_size=split)

In [47]:
X_train

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
83887,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,1.616978,-0.053906,-0.031767,-0.019726,-1.211901,-0.046432
105197,-0.110249,-0.007569,-0.004837,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,-0.618438,-0.053906,-0.031767,-0.019726,0.825150,-0.046432
131702,0.043450,0.577911,-0.096896,-0.017624,-0.059104,-0.019459,-0.113521,-0.143999,-0.890373,-0.016494,...,-0.453815,-0.18843,-0.009419,-0.174880,-0.313124,-0.030535,-0.025803,-0.105681,0.718027,-0.056997
140096,0.043450,0.577911,-0.096896,-0.017624,-0.059104,-0.019459,-0.113521,-0.143999,-0.890373,-0.016494,...,-0.453815,-0.18843,-0.009419,-0.174880,-0.313124,-0.030535,-0.025803,-0.105681,0.718027,-0.056997
134296,-0.155534,-0.021480,-0.067959,-0.017624,-0.059104,-0.019459,-0.113521,-0.143999,-0.890373,-0.016494,...,-0.453815,-0.18843,-0.009419,-0.174880,-0.313124,-0.030535,-0.025803,-0.105681,0.718027,-0.056997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59753,-0.109481,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,-0.312889,-0.11205,-0.028606,7.143771,-0.618438,-0.053906,-0.031767,-0.019726,-1.211901,-0.046432
3516,-0.110249,-0.007761,-0.004909,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,-0.618438,-0.053906,-0.031767,-0.019726,0.825150,-0.046432
138463,-0.155534,-0.021988,-0.094822,-0.017624,-0.059104,-0.019459,-0.113521,-0.143999,-0.890373,-0.016494,...,-0.453815,-0.18843,-0.009419,-0.174880,-0.313124,-0.030535,-0.025803,9.462460,-1.392705,-0.056997
52372,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,1.616978,-0.053906,-0.031767,-0.019726,-1.211901,-0.046432


In [48]:
y_train

83887     1
105197    0
131702    3
140096    3
134296    3
         ..
59753     2
3516      0
138463    1
52372     1
146819    0
Name: Label, Length: 103961, dtype: int64

## LEVEL 0 - Weak models - Base Learner

In [49]:
with open(output_file_name, "a") as f: print('------------START of WEAK LEARNERS (BASE MODELS) - STACK 00 -----------------', file = f)

#Defining Basemodels


print('---------------------------------------------------------------------------------')
print('Defining RF Model')
print('---------------------------------------------------------------------------------')
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
rf = RandomForestClassifier(max_depth = 5,  n_estimators = 10, min_samples_split = 2, n_jobs = -1)
#------------------------------------------------------------------------------


print('---------------------------------------------------------------------------------')
print('Defining ADA Model')
print('---------------------------------------------------------------------------------')
#ADA
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
import time
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)


print('---------------------------------------------------------------------------------')
print('Defining LGBM Model')
print('---------------------------------------------------------------------------------')
#LGBM
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()



#KNN
print('---------------------------------------------------------------------------------')
print('Defining KNN Model')
print('---------------------------------------------------------------------------------')
from sklearn.neighbors import KNeighborsClassifier
knn_clf=KNeighborsClassifier(n_neighbors = 5)


#SVM
print('---------------------------------------------------------------------------------')
print('Defining SVM Model')
print('---------------------------------------------------------------------------------')

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier

# Instantiate the SGDClassifier with additional hyperparameters
clf = SGDClassifier(
    loss='hinge',           # hinge loss for linear SVM
    penalty='l2',           # L2 regularization to prevent overfitting
    alpha=1e-4,             # Learning rate (small value for fine-grained updates)
    max_iter=1000,          # Number of passes over the training data
    random_state=42,        # Seed for reproducible results
    learning_rate='optimal' # Automatically adjusts the learning rate based on the training data
)


#MLP
print('---------------------------------------------------------------------------------')
print('Defining MLP Model')
print('---------------------------------------------------------------------------------')


from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
import time

# create MLPClassifier instance
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=1)


#DNN
print('---------------------------------------------------------------------------------')
print('Defining DNN Model')
print('---------------------------------------------------------------------------------')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# #Model Parameters
# dropout_rate = 0.01
# nodes = 70
# out_layer = 5
# optimizer='adam'
# loss='sparse_categorical_crossentropy'
# epochs=1
# batch_size=2*256

#Model Parameters
dropout_rate = 0.2
nodes = 3
out_layer = 5
optimizer='adam'
loss='sparse_categorical_crossentropy'
epochs=100
batch_size=128


num_columns = X_train.shape[1]

dnn = tf.keras.Sequential()

# Input layer
dnn.add(tf.keras.Input(shape=(num_columns,)))

# Dense layers with dropout
dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

# Output layer
dnn.add(tf.keras.layers.Dense(out_layer, activation='softmax'))

dnn.compile(optimizer=optimizer, loss=loss,metrics=['accuracy'])

dnn.summary()



# dnn = Sequential()
# dnn.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Input layer
# dnn.add(Dense(64, activation='relu'))  # Hidden layer
# dnn.add(Dense(5))  # Output layer

# dnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# # summary of model layers
# dnn.summary()

---------------------------------------------------------------------------------
Defining RF Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Defining ADA Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Defining LGBM Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Defining KNN Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Defining SVM Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Def

In [50]:
# #SVM
# # Wrap SGDClassifier with MultiOutputClassifier
# multi_target_clf = MultiOutputClassifier(clf)

# # Fit the model on the training data
# multi_target_clf.fit(X_train, y_train)

# Make predictions on the test data
# y_pred = clf.predict(X_test)



In [51]:
#Training Basemodels
import joblib
from sklearn.model_selection import StratifiedKFold, cross_val_score
n_splits = 5  # You can adjust the number of folds as needed



print('---------------------------------------------------------------------------------')
print('Training Model')
with open(output_file_name, "a") as f: print('Training weak models - level 0', file = f)

print('---------------------------------------------------------------------------------')

if use_model_ada == 1 and load_model_ada == 0:

    print('---------------------------------------------------------------------------------')
    print('Training ADA')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training ADA', file = f)
    print('---------------------------------------------------------------------------------')
    #ADA


    start = time.time()
    ada = abc.fit(X_train, y_train)
    end = time.time()

    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(ada, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)

    ada_tr_time_taken= time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)

    # Assuming 'model' is your trained model
    joblib.dump(ada, 'ada_base_model.joblib')


if use_model_rf == 1 and load_model_rf == 0:

    print('---------------------------------------------------------------------------------')
    print('Training RF')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)
    with open(output_file_name, "a") as f: print('Training RF', file = f)
    print('---------------------------------------------------------------------------------')
    #RF
    start = time.time()
    model_rf = rf.fit(X_train,y_train)
    end = time.time()

    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(model_rf, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    rf_tr_time_taken = time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(model_rf, 'rf_base_model.joblib')

if use_model_svm == 1 and load_model_svm == 0:

    print('---------------------------------------------------------------------------------')
    print('Training SVM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training SVM', file = f)
    print('---------------------------------------------------------------------------------')
    #SVM

    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    # clf.score(X_train, y_train)
    svm_tr_time_taken= time_taken = end - start

    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(clf, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(clf, 'svm_base_model.joblib')


if use_model_knn == 1 and load_model_knn == 0:

    #KNN
    print('---------------------------------------------------------------------------------')
    print('Training KNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training KNN', file = f)
    print('---------------------------------------------------------------------------------')
    start = time.time()
    knn_clf.fit(X_train,y_train)
    end = time.time()


    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(knn_clf, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    knn_tr_time_taken = time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(knn_clf, 'knn_base_model.joblib')


if use_model_lgbm == 1 and load_model_lgbm == 0:


    print('---------------------------------------------------------------------------------')
    print('Training LGBM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training LGBM', file = f)
    print('---------------------------------------------------------------------------------')
    start = time.time()
    lgbm.fit(X_train, y_train)
    end = time.time()

    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(lgbm, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)

    lgbm_tr_time_taken = time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(lgbm, 'lgbm_base_model.joblib')

if use_model_mlp == 1 and load_model_mlp == 0:


    print('---------------------------------------------------------------------------------')
    print('Training MLP')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training MLP', file = f)
    print('---------------------------------------------------------------------------------')

    start = time.time()
    MLP = mlp.fit(X_train, y_train)
    end = time.time()

    # Create the StratifiedKFold object
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Perform cross-validation
    cv_scores = cross_val_score(MLP, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # Print the cross-validation scores
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)

    mlp_tr_time_taken= time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(MLP, 'mlp_base_model.joblib')


if use_model_dnn == 1 and load_model_dnn == 0:
    from keras.callbacks import EarlyStopping

    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
    print('---------------------------------------------------------------------------------')
    print('Training DNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training DNN', file = f)
    print('---------------------------------------------------------------------------------')
    # Convert Y_test back to its original format
    # y_test = np.argmax(Y_test, axis=1)

    # Start the timer
    start = time.time()
    # dnn.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    dnn.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2, callbacks=[early_stopping])

    # End the timer
    end = time.time()

    # # Create the StratifiedKFold object
    # stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # # Perform cross-validation
    # cv_scores = cross_val_score(dnn, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # # Print the cross-validation scores
    # print("Cross-validation scores:", cv_scores)
    # print("Mean accuracy:", cv_scores.mean())
    # with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    dnn_tr_time_taken= time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    dnn.save("DNN_base_model.h5")

    # Calculate the time taken and print it out
    # print(f'Time taken for training: {time_taken} seconds')


with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)



---------------------------------------------------------------------------------
Training Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training ADA
---------------------------------------------------------------------------------


Cross-validation scores: [0.59779734 0.83060793 0.66116776 0.58839938 0.8088207 ]
Mean accuracy: 0.6973586206489453
---------------------------------------------------------------------------------
Training RF
---------------------------------------------------------------------------------
Cross-validation scores: [0.93911412 0.93035783 0.9547903  0.92992497 0.9348788 ]
Mean accuracy: 0.9378132059049159
---------------------------------------------------------------------------------
Training SVM
---------------------------------------------------------------------------------
Cross-validation scores: [0.97114414 0.9674875  0.96801654 0.97042132 0.96878607]
Mean accuracy: 0.9691711125035105
---------------------------------------------------------------------------------
Training KNN
---------------------------------------------------------------------------------


In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import StratifiedKFold

# # Define your Keras model as a function
# def create_model(optimizer='adam', hidden_layer_size=16):
#     # model = Sequential()
#     # model.add(Dense(hidden_layer_size, input_dim=input_size, activation='relu'))
#     # model.add(Dense(1, activation='sigmoid'))
#     # model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

        
#     dnn = tf.keras.Sequential()

#     # Input layer
#     dnn.add(tf.keras.Input(shape=(num_columns,)))

#     # Dense layers with dropout
#     dnn.add(tf.keras.layers.Dense(nodes))
#     dnn.add(tf.keras.layers.Dropout(dropout_rate))

#     dnn.add(tf.keras.layers.Dense(nodes))
#     dnn.add(tf.keras.layers.Dropout(dropout_rate))

#     dnn.add(tf.keras.layers.Dense(nodes))
#     dnn.add(tf.keras.layers.Dropout(dropout_rate))

#     dnn.add(tf.keras.layers.Dense(nodes))
#     dnn.add(tf.keras.layers.Dropout(dropout_rate))

#     dnn.add(tf.keras.layers.Dense(nodes))
#     dnn.add(tf.keras.layers.Dropout(dropout_rate))

#     # Output layer
#     dnn.add(tf.keras.layers.Dense(out_layer))



#     dnn.compile(optimizer=optimizer, loss=loss)

#     dnn.summary()
#     return dnn

# # Create a KerasClassifier
# dnn = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# # Define the parameter grid for GridSearchCV
# param_grid = {
#     'optimizer': ['adam', 'sgd'],
#     'hidden_layer_size': [8, 16, 32]
# }

# # Create the StratifiedKFold
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # Create GridSearchCV
# grid = GridSearchCV(estimator=dnn, param_grid=param_grid, cv=cv, scoring='accuracy')
# grid_result = grid.fit(X_train, y_train)

# # Print the best parameters and best accuracy
# print("Best Parameters: ", grid_result.best_params_)
# print("Best Accuracy: ", grid_result.best_score_)



In [ ]:
# stratified_kfold

In [ ]:
# Loading Models
from tensorflow.keras.models import load_model

if load_model_ada == 1:
    ada = joblib.load('ada_base_model.joblib')

if load_model_svm == 1:
    clf =  joblib.load('svm_base_model.joblib')

if load_model_dnn == 1:
    dnn = load_model("DNN_base_model.h5")

if load_model_knn == 1:
    knn_clf = joblib.load('knn_base_model.joblib')

if load_model_mlp == 1:
    MLP = joblib.load('mlp_base_model.joblib')

if load_model_rf == 1:
    rf = joblib.load('rf_base_model.joblib')

if load_model_lgbm == 1:
    lgbm = joblib.load('lgbm_base_model.joblib')







In [ ]:
# Make predictions on the test data
# preds_svm = clf.predict(X_test)



# y_scores = y_pred
# y_true = y_test



### Base leaners predictions

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
with open(output_file_name, "a") as f: print('Generating Predictions', file = f)

if use_model_rf == 1:

    print('---------------------------------------------------------------------------------')
    print('Prediction RF')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction RF', file = f)
    print('---------------------------------------------------------------------------------')
    #RF
    start = time.time()
    preds_rf = rf.predict(X_test)
    preds_rf_prob = rf.predict_proba(X_test)
    end = time.time()
    rf_pr_time_taken=  time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_svm == 1:

    print('Prediction SVM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction SVM', file = f)
    print('---------------------------------------------------------------------------------')
    #SVM
    start = time.time()
    preds_svm = clf.predict(X_test)
    # preds_svm_prob = clf.predict_proba(X_test)

    #Since SVM does not deal with prob by nature we use a meta learner
    # https://stackoverflow.com/questions/55250963/how-to-get-probabilities-for-sgdclassifier-linearsvm

    model = CalibratedClassifierCV(clf)

    model.fit(X, y)
    preds_svm_prob = model.predict_proba(X)

    end = time.time()
    svm_pr_time_taken = time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_lgbm == 1:

    print('Prediction LGBM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction LGBM', file = f)
    print('---------------------------------------------------------------------------------')
    #LGBM
    start = time.time()
    preds_lgbm = lgbm.predict(X_test)
    preds_lgbm_prob = lgbm.predict_proba(X_test)

    end = time.time()
    lgbm_pr_time_taken=time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_dnn == 1:

    print('Prediction DNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction DNN', file = f)
    print('---------------------------------------------------------------------------------')
    #DNN
    start = time.time()
    pred_dnn = dnn.predict(X_test)
    preds_dnn_prob = pred_dnn
    preds_dnn = np.argmax(pred_dnn,axis = 1)
    end = time.time()
    dnn_pr_time_taken=time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_ada == 1:

    print('Prediction ADA')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction ADA', file = f)
    print('---------------------------------------------------------------------------------')
    #ADA
    start = time.time()
    preds_ada = ada.predict(X_test)
    preds_ada_prob = ada.predict_proba(X_test)

    end = time.time()
    ada_pr_time_taken=time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')
    print('Prediction MLP')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction MLP', file = f)
    print('---------------------------------------------------------------------------------')

if use_model_mlp == 1:

    #MLP
    start = time.time()
    y_pred = MLP.predict_proba(X_test)
    preds_mlp_prob = y_pred
    preds_mlp = np.argmax(y_pred,axis = 1)
    end = time.time()
    mlp_pr_time_taken=time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')
    print('Prediction KNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction KNN', file = f)
    print('---------------------------------------------------------------------------------')

if use_model_knn == 1:

    #KNN
    start = time.time()
    preds_knn =knn_clf.predict(X_test)
    preds_knn_prob =knn_clf.predict_proba(X_test)

    preds_knn
    end = time.time()
    knn_pr_time_taken=time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)


---------------------------------------------------------------------------------
Prediction RF
---------------------------------------------------------------------------------


---------------------------------------------------------------------------------
Prediction SVM
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction LGBM
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction DNN
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction ADA
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction MLP
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction KNN
-------

In [ ]:
# from sklearn.calibration import CalibratedClassifierCV
# model = CalibratedClassifierCV(clf)

# model.fit(X, y)
# preds_svm_prob = model.predict_proba(X)

# print(preds_ada_prob)
# print(preds_knn_prob)
# print(preds_dnn_prob)
# print(preds_mlp_prob)
# print(preds_rf_prob)
# print(preds_svm_prob)


In [ ]:
print(preds_svm_prob)
preds_3 = np.argmax(preds_svm_prob,axis = 1)
print(preds_3)

print(preds_svm)
# print(y_train)

[[7.86755451e-01 9.15808640e-02 7.78774782e-02 4.31012582e-02
  6.84948135e-04]
 [2.62305098e-01 2.71329463e-02 6.45857927e-01 6.23936481e-02
  2.31038051e-03]
 [1.69951428e-02 9.10446456e-01 5.10282199e-02 2.14799179e-02
  5.02632551e-05]
 ...
 [9.42972082e-01 3.31526500e-02 1.34223706e-03 2.24056309e-02
  1.27400343e-04]
 [9.12741440e-01 3.97164126e-02 1.80198033e-02 2.95111286e-02
  1.12159303e-05]
 [5.57736620e-01 3.72736907e-01 5.55977328e-02 1.35376604e-02
  3.91079556e-04]]
[0 2 1 ... 0 0 0]
[1 0 0 ... 1 1 0]


### METRICS - Base Learners

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score



# >>> 
# >>> roc_auc_score(y, clf.predict_proba(X)[:, 1])
# 0.99...
# >>> roc_auc_score(y, clf.decision_function(X))

#### RF

In [ ]:
# y_test
# pred_label

In [ ]:
#RF
if use_model_rf == 1:
    # start = time.time()
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('RF base model', file = f)

    pred_label = preds_rf
    name = 'rf'
    metrics = confusion_metrics(name, pred_label, y_test)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    globals()[f"{name}_time_00"] = rf_pr_time_taken + rf_tr_time_taken


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2    3    4
0  22616.0    169.0   220.0  2.0  0.0
1    964.0  15042.0    95.0  0.0  0.0
2    350.0    292.0  3635.0  0.0  0.0
3   1009.0      1.0   122.0  6.0  0.0
4     32.0      0.0     1.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9269009785438549
Precision total:  0.7036708795596386
Recall total:  0.5544800107715953
F1 total:  0.5551540048700508
BACC total:  0.5544800107715953
MCC total:  0.875668923834827


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [ ]:
#DNN
if use_model_dnn == 1:
    start = time.time()
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('DNN base model', file = f)


    pred_label = preds_dnn
    name = 'dnn'
    metrics = confusion_metrics(name, pred_label, y_test)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = dnn_pr_time_taken + dnn_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1    2    3    4
0  22693.0    314.0  0.0  0.0  0.0
1   1382.0  14719.0  0.0  0.0  0.0
2   2101.0   2176.0  0.0  0.0  0.0
3   1128.0     10.0  0.0  0.0  0.0
4     29.0      4.0  0.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.8396624472573839
Precision total:  0.3369709699388192
Recall total:  0.38010376035375015
F1 total:  0.35699530838837495
BACC total:  0.38010376035375015
MCC total:  0.7226654496793403


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [ ]:
#ADA
if use_model_ada == 1:
    start = time.time()
    
    pred_label = preds_ada
    name = 'ada'
    metrics = confusion_metrics(name, pred_label, y_test)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = ada_pr_time_taken + ada_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3    4
0  18156.0   2883.0  1462.0  505.0  1.0
1    251.0  12405.0  3356.0   89.0  0.0
2    426.0   1050.0  2743.0   58.0  0.0
3    555.0     57.0    72.0  446.0  8.0
4     12.0      0.0     1.0   20.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------


Accuracy total:  0.7574737409103152
Precision total:  0.4901499320639549
Recall total:  0.5185706391688746
F1 total:  0.4951450071968294
BACC total:  0.5185706391688746
MCC total:  0.6186463514073031


In [ ]:
#SVM
if use_model_svm == 1:
    start = time.time()

    pred_label = preds_svm
    name = 'svm'
    metrics = confusion_metrics(name, pred_label, y_test)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = svm_pr_time_taken + svm_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2    3    4
0  21396.0    492.0  1114.0  5.0  0.0
1   1283.0  14671.0   147.0  0.0  0.0
2    521.0   1198.0  2558.0  0.0  0.0
3    970.0     36.0   131.0  1.0  0.0
4     27.0      3.0     3.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------


Accuracy total:  0.866909058263758
Precision total:  0.518517030256721
Recall total:  0.4880249952789204
F1 total:  0.48654255532153956
BACC total:  0.4880249952789204
MCC total:  0.7710117391455595


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [ ]:
#KNN
if use_model_knn == 1:
    start = time.time()
    pred_label = preds_knn
    name = 'knn'
    metrics = confusion_metrics(name, pred_label, y_test)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = knn_pr_time_taken + knn_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3    4
0  22464.0    186.0   163.0  194.0  0.0
1    114.0  15957.0    17.0   12.0  1.0
2    144.0     22.0  4092.0   19.0  0.0
3    133.0      4.0    10.0  991.0  0.0
4     24.0      2.0     0.0    6.0  1.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------


Accuracy total:  0.9764117066163929
Precision total:  0.8470442740595739
Recall total:  0.7650658733325508
F1 total:  0.7642377601210751
BACC total:  0.7650658733325508
MCC total:  0.9602870072104996


In [ ]:
#MLP
if use_model_mlp == 1:
    start = time.time()
    pred_label = preds_mlp
    name = 'mlp'
    metrics = confusion_metrics(name, pred_label, y_test)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = mlp_pr_time_taken + mlp_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3    4
0  22293.0    159.0   298.0  257.0  0.0
1    252.0  15788.0    42.0   19.0  0.0
2    148.0     46.0  4056.0   27.0  0.0
3    176.0     16.0    14.0  932.0  0.0
4     25.0      2.0     0.0    6.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------


Accuracy total:  0.9666262680671515
Precision total:  0.7261111598469259
Recall total:  0.7433670231692975
F1 total:  0.734398287148619
BACC total:  0.7433670231692975
MCC total:  0.9438970625968045


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [ ]:
#lgbm
start_lgbm = time.time()
if use_model_lgbm == 1:

    pred_label = preds_lgbm
    name = 'lgbm'
    metrics = confusion_metrics(name, pred_label, y_test)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start_lgbm
    globals()[f"{name}_time_00"] = lgbm_pr_time_taken + lgbm_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3     4
0  22512.0    136.0   144.0  170.0  45.0
1    191.0  15841.0    36.0   21.0  12.0
2    113.0     21.0  4119.0   15.0   9.0
3    119.0      0.0    17.0  989.0  13.0
4     20.0      0.0     0.0    9.0   4.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------


Accuracy total:  0.9755139599604992
Precision total:  0.7589729746251801
Recall total:  0.7831351249047284
F1 total:  0.7677661766112274
BACC total:  0.7831351249047284
MCC total:  0.9588050806562161


### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
start = time.time()

# Create a Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)
# Train the classifier on the training data
dt_classifier.fit(X_train, y_train)
# Make predictions on the test data
preds_dt = dt_classifier.predict(X_test)
# Evaluate the accuracy of the model
preds_dt_prob = dt_classifier.predict_proba(X_test)


pred_label = preds_dt
name = 'dt'
metrics = confusion_metrics(name, pred_label, y_test)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    

globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC
end = time.time()
time_taken = end - start
globals()[f"{name}_time_00"] = time_taken

### CATBOOST

In [ ]:
import catboost
start = time.time()
cat_00 = catboost.CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass', custom_metric='Accuracy')

# Fit the model
cat_00.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=10)

# Make predictions on the test set
preds_cat = cat_00.predict(X_test)
preds_cat_prob = cat_00.predict_proba(X_test)
preds_cat = np.squeeze(preds_cat)


if 1 == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Catboost base model', file = f)


    

    pred_label = preds_cat
    
    

    # pred_label = y_pred

    name = 'cat'
    metrics = confusion_metrics(name, pred_label, y_test)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = time_taken


0:	learn: 1.3044871	test: 1.3039134	best: 1.3039134 (0)	total: 78.1ms	remaining: 7.73s


10:	learn: 0.4630357	test: 0.4618663	best: 0.4618663 (10)	total: 326ms	remaining: 2.64s
20:	learn: 0.2676941	test: 0.2670392	best: 0.2670392 (20)	total: 558ms	remaining: 2.1s
30:	learn: 0.1908402	test: 0.1910453	best: 0.1910453 (30)	total: 814ms	remaining: 1.81s
40:	learn: 0.1529783	test: 0.1538367	best: 0.1538367 (40)	total: 1.05s	remaining: 1.52s
50:	learn: 0.1353746	test: 0.1365274	best: 0.1365274 (50)	total: 1.27s	remaining: 1.22s
60:	learn: 0.1221059	test: 0.1237006	best: 0.1237006 (60)	total: 1.49s	remaining: 949ms
70:	learn: 0.1126627	test: 0.1146082	best: 0.1146082 (70)	total: 1.68s	remaining: 688ms
80:	learn: 0.1054775	test: 0.1077428	best: 0.1077428 (80)	total: 1.92s	remaining: 449ms
90:	learn: 0.0997292	test: 0.1022796	best: 0.1022796 (90)	total: 2.14s	remaining: 212ms
99:	learn: 0.0950370	test: 0.0978098	best: 0.0978098 (99)	total: 2.35s	remaining: 0us

bestTest = 0.09780975775
bestIteration = 99

-----------------------------------------------------------------------------

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [ ]:
import xgboost as xgb
start = time.time()
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# Assuming you have your features and labels as X and y
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set XGBoost parameters
params = {
    'objective': 'multi:softmax',  # for multi-class classification
    'num_class': 5,  # specify the number of classes
    'max_depth': 3,
    'learning_rate': 0.1,
    'eval_metric': 'mlogloss'  # metric for multi-class classification
}

# Train the XGBoost model
num_round = 100
xgb_00 = xgb.train(params, dtrain, num_round)

# Make predictions on the test set
preds_xgb = xgb_00.predict(dtest)
# preds_xgb_prob = xgb_00.predict_proba(dtest)


# Get class probabilities
# Assuming binary classification, get the probability for the positive class (class 1)
preds_xgb_margin = xgb_00.predict(dtest, output_margin=True)
preds_xgb_prob = 1 / (1 + np.exp(-preds_xgb_margin))

# Print or use positive_class_probabilities as needed
# print(positive_class_probabilities)


# Convert predicted probabilities to class labels (if necessary)
# y_pred_labels = [round(value) for value in y_pred]

# Evaluate the accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))


In [ ]:

if 1 == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('xgboost base model', file = f)




    pred_label = preds_xgb
    # pred_label = label[ypred]
    name = 'xgb'
    metrics = confusion_metrics(name, pred_label, y_test)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0.0      1.0     2.0    3.0  4.0
0.0  22299.0    235.0   302.0  171.0  0.0
1.0    477.0  15502.0   103.0   19.0  0.0
2.0    212.0     88.0  3944.0   33.0  0.0
3.0    433.0      5.0    25.0  675.0  0.0
4.0     14.0      3.0     0.0   16.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.952060328575276
Precision total:  0.7141647286115903
Recall total:  0.6894623313800716
F1 total:  0.700173767262395
BACC total:  0.6894623313800716
MCC total:  0.9188887582697526


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### LR

In [ ]:
from sklearn.linear_model import LogisticRegression

#Logistic Regression
print('---------------------------------------------------------------------------------')
print('Defining Logistic Regression Model')
print('---------------------------------------------------------------------------------')
logreg_00 = LogisticRegression()

if 1 == 1 and 0 == 0:
    print('---------------------------------------------------------------------------------')
    print('Training LR ')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training LR', file = f)
    print('---------------------------------------------------------------------------------')
    start_lr = start = time.time()
    logreg_00.fit(X_train,y_train)
    end = time.time()


    # # Create the StratifiedKFold object
    # stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # # Perform cross-validation
    # cv_scores = cross_val_score(knn_clf, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
    # # Print the cross-validation scores
    # print("Cross-validation scores:", cv_scores)
    # print("Mean accuracy:", cv_scores.mean())
    # with open(output_file_name, "a") as f: print('mean accuracy', cv_scores.mean() , file = f)


    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    # joblib.dump(logreg_01, 'logreg_01.joblib')


# if 1 == 1:
    # logreg_01 = joblib.load('logreg_01.joblib')

if 1 == 1:

    #lR
    start = time.time()
    preds_lr = preds_logreg =logreg_00.predict(X_test)
    preds_lr_prob = logreg_00.predict_proba(X_test)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#LR
if 1 == 1:
    pred_label = preds_logreg
    name = 'lr'
    metrics = confusion_metrics(name, pred_label, y_test)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    
    end = time.time()
    time_taken = end - start_lr
    globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
Defining Logistic Regression Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training LR 
---------------------------------------------------------------------------------


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2     3    4
0  21419.0    546.0   992.0  50.0  0.0
1   1274.0  14749.0    78.0   0.0  0.0
2    976.0   1216.0  2085.0   0.0  0.0
3    971.0     27.0   127.0  13.0  0.0
4     24.0      5.0     3.0   1.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.8588293383607146
Precision total:  0.51956305186457
Recall total:  0.4691844742165842
F1 total:  0.47506332958775077
BACC total:  0.4691844742165842
MCC total:  0.7557912792195774


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### Bagging DT  


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
start = time.time()
# # Define the base classifier (Decision Tree in this case)
base_classifier = DecisionTreeClassifier(random_state=42)

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

# Evaluate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy}')

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)


pred_label = y_pred
name = 'bag_dt'
metrics = confusion_metrics(name, pred_label, y_test)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    

globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC
end = time.time()
time_taken = end - start
globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3    4
0  22613.0    133.0   107.0  150.0  4.0
1    157.0  15921.0    18.0    5.0  0.0
2    122.0     23.0  4118.0   14.0  0.0
3    138.0      2.0    11.0  979.0  8.0
4     20.0      0.0     0.0    8.0  5.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9793518269144448
Precision total:  0.8160496565078939
Recall total:  0.7892632207239577
F1 total:  0.7980809808386418
BACC total:  0.7892632207239577
MCC total:  0.9651604098104272


## bagging SVM

In [ ]:
## bagging  with SVM
from sklearn.ensemble import BaggingClassifier
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

start = time.time()

from sklearn.linear_model import SGDClassifier

# Instantiate the SGDClassifier with additional hyperparameters
svm_01 = SGDClassifier(
    loss='hinge',           # hinge loss for linear SVM
    penalty='l2',           # L2 regularization to prevent overfitting
    alpha=1e-4,             # Learning rate (small value for fine-grained updates)
    max_iter=1000,          # Number of passes over the training data
    random_state=42,        # Seed for reproducible results
    learning_rate='optimal' # Automatically adjusts the learning rate based on the training data
)

# # Define the base classifier (Decision Tree in this case)
base_classifier = svm_01

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)


with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_svm'
pred_label = y_pred
metrics = confusion_metrics(name, pred_label, y_test)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC

end = time.time()
time_taken = end - start
globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2    3    4
0  21394.0    549.0  1061.0  3.0  0.0
1   1279.0  14689.0   133.0  0.0  0.0
2    603.0   1200.0  2474.0  0.0  0.0
3    970.0     37.0   129.0  2.0  0.0
4     27.0      3.0     3.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.865405332615136
Precision total:  0.5647748952564643
Recall total:  0.48447895788066797
F1 total:  0.4845682631281675
BACC total:  0.48447895788066797
MCC total:  0.7681012409166899


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


## Bagging MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
import time
start = time.time()
# create MLPClassifier instance
mlp_00 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=1)

base_classifier = mlp_00

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

# Evaluate accuracy
# accuracy = accuracy_score(y_test_00, y_pred)
# print(f'Accuracy: {accuracy}')

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_mlp'
pred_label = y_pred
metrics = confusion_metrics(name, pred_label, y_test)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC
end = time.time()
time_taken = end - start
globals()[f"{name}_time_00"] = time_taken


Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3    4
0  22334.0    176.0   309.0  188.0  0.0
1    192.0  15842.0    52.0   15.0  0.0
2    123.0     21.0  4111.0   22.0  0.0
3    228.0      1.0    21.0  888.0  0.0
4     23.0      1.0     0.0    9.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9690052967052698
Precision total:  0.7338601677386349
Recall total:  0.7392332337398194
F1 total:  0.7364232688402375
BACC total:  0.7392332337398194
MCC total:  0.9478644629569626


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### bagging KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_00=KNeighborsClassifier(n_neighbors = 5)
start = time.time()
base_classifier = knn_00

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

# Evaluate accuracy
# accuracy = accuracy_score(y_test_00, y_pred)
# print(f'Accuracy: {accuracy}')

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_knn'

pred_label = y_pred


metrics = confusion_metrics(name, pred_label, y_test)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC
end = time.time()
time_taken = end - start
globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3    4
0  22459.0    182.0   166.0  198.0  2.0
1    116.0  15950.0    22.0   12.0  1.0
2    129.0     17.0  4110.0   21.0  0.0
3    131.0      3.0    11.0  991.0  2.0
4     21.0      2.0     0.0    9.0  1.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.976546368614777
Precision total:  0.7791045487524741
Recall total:  0.7657771686591303
F1 total:  0.7627367120222106
BACC total:  0.7657771686591303
MCC total:  0.9605401288763482



### bag LogRegression


In [ ]:
from sklearn.linear_model import LogisticRegression
start = time.time()
#Logistic Regression
print('---------------------------------------------------------------------------------')
print('Defining baggin Logistic Regression Model')
print('---------------------------------------------------------------------------------')
logreg_00 = LogisticRegression()


base_classifier = logreg_00

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

# Evaluate accuracy
# accuracy = accuracy_score(y_test_00, y_pred)
# print(f'Accuracy: {accuracy}')

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_lr'

pred_label = y_pred


metrics = confusion_metrics(name, pred_label, y_test)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC
end = time.time()
time_taken = end - start
globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
Defining baggin Logistic Regression Model
---------------------------------------------------------------------------------


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2     3    4
0  21419.0    546.0   993.0  49.0  0.0
1   1273.0  14749.0    79.0   0.0  0.0
2    968.0   1216.0  2093.0   0.0  0.0
3    971.0     27.0   127.0  13.0  0.0
4     24.0      5.0     3.0   1.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.8590088876918933
Precision total:  0.5203716270654932
Recall total:  0.46955856820769953
F1 total:  0.47537782565924197
BACC total:  0.46955856820769953
MCC total:  0.7561206279709384


### Bagging RF

In [ ]:
start = time.time()

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth = 5,  n_estimators = 10, min_samples_split = 2, n_jobs = -1)

base_classifier = rf

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_rf'

pred_label = y_pred


metrics = confusion_metrics(name, pred_label, y_test)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC
end = time.time()
time_taken = end - start
globals()[f"{name}_time_00"] = time_taken



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2     3    4
0  22639.0    148.0   216.0   4.0  0.0
1    992.0  15039.0    70.0   0.0  0.0
2    381.0    191.0  3705.0   0.0  0.0
3   1075.0      0.0    24.0  39.0  0.0
4     32.0      0.0     0.0   1.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9296615495107281
Precision total:  0.737675730295928
Recall total:  0.5637156560827961
F1 total:  0.5711876824692197
BACC total:  0.5637156560827961
MCC total:  0.8806680889666707


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### Bagging ADA

In [ ]:
start = time.time()

from sklearn.ensemble import AdaBoostClassifier
import time
ada = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)

base_classifier = ada

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_ada'

pred_label = y_pred


metrics = confusion_metrics(name, pred_label, y_test)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC
end = time.time()
time_taken = end - start
globals()[f"{name}_time_00"] = time_taken


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3     4
0  17016.0   4739.0   964.0  273.0  15.0
1    417.0  12639.0  2980.0   63.0   2.0
2    545.0   1028.0  2673.0   31.0   0.0
3    522.0     40.0     5.0  567.0   4.0
4     11.0      0.0     1.0   18.0   3.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.7383517371397792
Precision total:  0.5457216368368039
Recall total:  0.5477411371621115
F1 total:  0.537939422804649
BACC total:  0.5477411371621115
MCC total:  0.586453467150978


### Bagging LGBM

In [ ]:
start = time.time()

from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()


base_classifier = lgbm

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_lgbm'

pred_label = y_pred


metrics = confusion_metrics(name, pred_label, y_test)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC
end = time.time()
time_taken = end - start
globals()[f"{name}_time_00"] = time_taken



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
         0        1       2      3    4
0  22588.0    145.0   123.0  147.0  4.0
1     97.0  15973.0    22.0    9.0  0.0
2    108.0     11.0  4148.0   10.0  0.0
3    128.0      1.0    13.0  990.0  6.0
4     20.0      0.0     0.0   11.0  2.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.980810665230272
Precision total:  0.7906364030266764
Recall total:  0.774846068629134
F1 total:  0.7777617691423482
BACC total:  0.774846068629134
MCC total:  0.9676636808145522


### Bagging Catboost 

In [ ]:
import catboost
start = time.time()

bag_cat = catboost.CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass', custom_metric='Accuracy')

base_classifier = bag_cat

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_cat'

pred_label = y_pred


metrics = confusion_metrics(name, pred_label, y_test)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC
end = time.time()
time_taken = end - start
globals()[f"{name}_time_00"] = time_taken



0:	learn: 1.3066352	total: 28.8ms	remaining: 2.85s
1:	learn: 1.1146871	total: 52.4ms	remaining: 2.57s
2:	learn: 0.9747702	total: 74ms	remaining: 2.39s
3:	learn: 0.8682439	total: 93.4ms	remaining: 2.24s
4:	learn: 0.7769188	total: 116ms	remaining: 2.2s
5:	learn: 0.7031324	total: 141ms	remaining: 2.21s
6:	learn: 0.6427024	total: 167ms	remaining: 2.22s
7:	learn: 0.5875422	total: 191ms	remaining: 2.19s
8:	learn: 0.5432293	total: 215ms	remaining: 2.18s
9:	learn: 0.5019902	total: 242ms	remaining: 2.17s
10:	learn: 0.4668573	total: 269ms	remaining: 2.18s
11:	learn: 0.4357264	total: 294ms	remaining: 2.16s
12:	learn: 0.4085990	total: 320ms	remaining: 2.14s
13:	learn: 0.3833979	total: 345ms	remaining: 2.12s
14:	learn: 0.3617594	total: 370ms	remaining: 2.1s
15:	learn: 0.3421378	total: 397ms	remaining: 2.09s
16:	learn: 0.3250150	total: 424ms	remaining: 2.07s
17:	learn: 0.3087622	total: 450ms	remaining: 2.05s
18:	learn: 0.2957416	total: 473ms	remaining: 2.02s
19:	learn: 0.2820295	total: 498ms	remaini

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### Bagging Combined

In [ ]:
### Bagging with many models
##### do bootstrapping 
##### 1. Multiple subsets are created from the original dataset, selecting observations with replacement.

start = time.time()

num_bootstraps = 10  # Adjust the number of bootstraps as needed

original_data_df = X_train.assign(label = y_train)
boot_df = []
for i in range(0,num_bootstraps): 
    boot_df.append(original_data_df.sample(frac = 1, replace=True).reset_index(drop=True))

# boot_df[5]

#### 2.A base model (weak model) is created on each of these subsets.
bag_comb_pred = []

# SVM
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(
    loss='hinge',           # hinge loss for linear SVM
    penalty='l2',           # L2 regularization to prevent overfitting
    alpha=1e-4,             # Learning rate (small value for fine-grained updates)
    max_iter=1000,          # Number of passes over the training data
    random_state=42,        # Seed for reproducible results
    learning_rate='optimal' # Automatically adjusts the learning rate based on the training data
)
y_train_boot = boot_df[0].pop('label')
X_train_boot = boot_df[0]
clf.fit(X_train_boot, y_train_boot)
preds_svm_00 = clf.predict(X_test)
bag_comb_pred.append(preds_svm_00)




#ADA
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)
ada = abc.fit(X_train, y_train)
y_train_boot = boot_df[1].pop('label')
X_train_boot = boot_df[1]
preds_ada_00 = ada.predict(X_test)
bag_comb_pred.append(preds_ada_00)

#Catboost
import catboost
cat_00 = catboost.CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass', custom_metric='Accuracy')
y_train_boot = boot_df[2].pop('label')
X_train_boot = boot_df[2]
cat_00.fit(X_train_boot, y_train_boot, eval_set=(X_test, y_test), verbose=10)
preds_cat = cat_00.predict(X_test)
preds_cat = np.squeeze(preds_cat)
pred_label = preds_cat
bag_comb_pred.append(preds_cat)

#MLP
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=1)
y_train_boot = boot_df[3].pop('label')
X_train_boot = boot_df[3]
if 1 == 1 and 0 == 0:
    MLP = mlp.fit(X_train_boot, y_train_boot)
    y_pred = MLP.predict_proba(X_test)
    preds_mlp_00 = np.argmax(y_pred,axis = 1)

bag_comb_pred.append(preds_mlp_00)

#LGBM
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()
y_train_boot = boot_df[4].pop('label')
X_train_boot = boot_df[4]

if 1 == 1 and 0 == 0:
    lgbm.fit(X_train_boot, y_train_boot)
    preds_lgbm_00 = lgbm.predict(X_test)
    bag_comb_pred.append(preds_lgbm_00)
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn_clf_00=KNeighborsClassifier(n_neighbors = 5)
y_train_boot = boot_df[5].pop('label')
X_train_boot = boot_df[5]

if 1 == 1 and 0 == 0:
    knn_clf_00.fit(X_train_boot,y_train_boot)
if use_model_knn == 1:
    preds_knn =knn_clf_00.predict(X_test)
    bag_comb_pred.append(preds_knn)
#Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth = 5,  n_estimators = 10, min_samples_split = 2, n_jobs = -1)
y_train_boot = boot_df[6].pop('label')
X_train_boot = boot_df[6]

if True == True:
    model_rf_00 = rf.fit(X_train_boot,y_train_boot)
    preds_rf_00 = model_rf_00.predict(X_test)
    bag_comb_pred.append(preds_rf_00)
#DNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#Model Parameters
y_train_boot = boot_df[7].pop('label')
X_train_boot = boot_df[7]


dropout_rate = 0.2
nodes = 3
out_layer = 5
optimizer='adam'
loss='sparse_categorical_crossentropy'
epochs=100
batch_size=128
num_columns = X_train_boot.shape[1]
dnn_00 = tf.keras.Sequential()
# Input layer
dnn_00.add(tf.keras.Input(shape=(num_columns,)))
# Dense layers with dropout
dnn_00.add(tf.keras.layers.Dense(nodes))
dnn_00.add(tf.keras.layers.Dropout(dropout_rate))
dnn_00.add(tf.keras.layers.Dense(nodes))
dnn_00.add(tf.keras.layers.Dropout(dropout_rate))
dnn_00.add(tf.keras.layers.Dense(nodes))
dnn_00.add(tf.keras.layers.Dropout(dropout_rate))
dnn_00.add(tf.keras.layers.Dense(nodes))
dnn_00.add(tf.keras.layers.Dropout(dropout_rate))
dnn_00.add(tf.keras.layers.Dense(nodes))
dnn_00.add(tf.keras.layers.Dropout(dropout_rate))
# Output layer
# dnn_00.add(tf.keras.layers.Dense(out_layer))
dnn_00.add(tf.keras.layers.Dense(out_layer, activation='softmax'))
dnn_00.compile(optimizer=optimizer, loss=loss,metrics=['accuracy'])
from keras.callbacks import EarlyStopping
# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
dnn_00.fit(X_train_boot, y_train_boot, epochs=epochs, batch_size=batch_size,validation_split=0.2, callbacks=[early_stopping])
pred_dnn = dnn_00.predict(X_test)
preds_dnn_00 = np.argmax(pred_dnn,axis = 1)
bag_comb_pred.append(preds_dnn_00)
#LogReg
from sklearn.linear_model import LogisticRegression
logreg_00 = LogisticRegression()
y_train_boot = boot_df[8].pop('label')
X_train_boot = boot_df[8]

logreg_00.fit(X_train_boot,y_train_boot)
preds_logreg =logreg_00.predict(X_test)
bag_comb_pred.append(preds_logreg)
import xgboost as xgb
y_train_boot = boot_df[9].pop('label')
X_train_boot = boot_df[9]

# Create a DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train_boot, label=y_train_boot)
dtest = xgb.DMatrix(X_test, label=y_test)
# Set XGBoost parameters
params = {
    'objective': 'multi:softmax',  # for multi-class classification
    'num_class': 5,  # specify the number of classes
    'max_depth': 3,
    'learning_rate': 0.1,
    'eval_metric': 'mlogloss'  # metric for multi-class classification
}
# Train the XGBoost model
num_round = 100
xgb_00 = xgb.train(params, dtrain, num_round)
preds_xgb_00 = xgb_00.predict(dtest)
bag_comb_pred.append(preds_xgb_00)
### 3. The models run in parallel and are independent of each other.
bag_vot_df = pd.DataFrame()
for i in range(0,len(bag_comb_pred)):
    bag_vot_df[f'model_{i}'] =  bag_comb_pred[i]
print(bag_vot_df)
# Voting start
from scipy.stats import mode
# bag_comb_pred_df = pd.DataFrame(bag_comb_pred)
# Extract predictions columns

# predictions = df[['dnn', 'rf', 'lgbm', 'ada', 'knn', 'mlp', 'svm','cat','xgb']]
    # selected_columns = df.loc[:, ~df.columns.isin(['rf'])]
predictions = bag_vot_df 

# predictions = bag_comb_pred_df.loc[:, ~df.columns.isin(['label'])] #df[column_features]

# Use the mode function along axis 1 to get the most common prediction for each row
ensemble_predictions, _ = mode(predictions.values, axis=1)

# Add the ensemble predictions to the DataFrame
bag_vot_df['ensemble'] = ensemble_predictions.astype(int)

# Display the DataFrame with ensemble predictions
print(bag_vot_df)

pred_label = bag_vot_df ['ensemble'].values
bag_vot_df.pop('ensemble')


name='bag_comb'
metrics = confusion_metrics(name, pred_label, y_test)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC
end = time.time()
time_taken = end - start
globals()[f"{name}_time_00"] = time_taken



0:	learn: 1.3049306	test: 1.3039542	best: 1.3039542 (0)	total: 29.3ms	remaining: 2.9s
10:	learn: 0.4641097	test: 0.4634997	best: 0.4634997 (10)	total: 286ms	remaining: 2.31s
20:	learn: 0.2651970	test: 0.2660280	best: 0.2660280 (20)	total: 527ms	remaining: 1.98s
30:	learn: 0.1914871	test: 0.1932211	best: 0.1932211 (30)	total: 753ms	remaining: 1.68s
40:	learn: 0.1547691	test: 0.1573027	best: 0.1573027 (40)	total: 1s	remaining: 1.44s
50:	learn: 0.1365355	test: 0.1395252	best: 0.1395252 (50)	total: 1.22s	remaining: 1.18s
60:	learn: 0.1222282	test: 0.1256268	best: 0.1256268 (60)	total: 1.43s	remaining: 915ms
70:	learn: 0.1114527	test: 0.1153250	best: 0.1153250 (70)	total: 1.66s	remaining: 680ms
80:	learn: 0.1043892	test: 0.1085262	best: 0.1085262 (80)	total: 1.89s	remaining: 442ms
90:	learn: 0.0984946	test: 0.1030729	best: 0.1030729 (90)	total: 2.11s	remaining: 209ms
99:	learn: 0.0933431	test: 0.0984391	best: 0.0984391 (99)	total: 2.33s	remaining: 0us

bestTest = 0.09843905549
bestIteration

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.


Epoch 1/100
650/650 [==============================] - 2s 3ms/step - loss: 1.7359 - accuracy: 0.5281 - val_loss: 0.6717 - val_accuracy: 0.8226
Epoch 2/100
650/650 [==============================] - 2s 3ms/step - loss: 0.8476 - accuracy: 0.7213 - val_loss: 0.5754 - val_accuracy: 0.8253
Epoch 3/100
650/650 [==============================] - 2s 3ms/step - loss: 0.7461 - accuracy: 0.7563 - val_loss: 0.5403 - val_accuracy: 0.8283
Epoch 4/100
650/650 [==============================] - 2s 3ms/step - loss: 0.6896 - accuracy: 0.7791 - val_loss: 0.5245 - val_accuracy: 0.8283
Epoch 5/100
650/650 [==============================] - 2s 3ms/step - loss: 0.6467 - accuracy: 0.7970 - val_loss: 0.5173 - val_accuracy: 0.8293
Epoch 6/100
650/650 [==============================] - 2s 3ms/step - loss: 0.6266 - accuracy: 0.8059 - val_loss: 0.5167 - val_accuracy: 0.8293
Epoch 7/100
650/650 [==============================] - 2s 3ms/step - loss: 0.6178 - accuracy: 0.8104 - val_loss: 0.5167 - val_accuracy: 0.8293

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


       model_0  model_1  model_2  model_3  model_4  model_5  model_6  model_7  \
0            1        1        1        1        1        1        1        1   
1            0        1        0        0        0        0        0        0   
2            0        0        0        0        0        0        0        0   
3            0        1        0        0        0        0        0        0   
4            1        1        1        1        1        1        1        1   
...        ...      ...      ...      ...      ...      ...      ...      ...   
44551        0        0        0        0        0        0        0        0   
44552        0        0        0        0        0        0        0        0   
44553        1        1        1        1        1        1        1        1   
44554        1        2        2        2        2        2        2        1   
44555        0        0        0        0        0        0        0        0   

       model_8  model_9  
0

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


## Creating new dataset for level 01

In [ ]:
print(len(preds_dnn_prob), len(y_test))

44556 44556


In [ ]:
print(y_test)

116749    1
12479     0
116681    0
122439    0
11327     1
         ..
125765    0
85435     0
34168     1
102278    2
47605     0
Name: Label, Length: 44556, dtype: int64


In [ ]:
df_from_series = y_test.to_frame()
y_test_reset_index = df_from_series.reset_index()
# y_test2 = y_test.reset_index(inplace=True)
print(y_test_reset_index)
y_test_reset_index.pop('index')

        index  Label
0      116749      1
1       12479      0
2      116681      0
3      122439      0
4       11327      1
...       ...    ...
44551  125765      0
44552   85435      0
44553   34168      1
44554  102278      2
44555   47605      0

[44556 rows x 2 columns]


0        116749
1         12479
2        116681
3        122439
4         11327
          ...  
44551    125765
44552     85435
44553     34168
44554    102278
44555     47605
Name: index, Length: 44556, dtype: int64

In [ ]:
y_test_reset_index.values[0][0]

1

In [ ]:
preds_dnn_2 = []
preds_svm_2 = []
preds_rf_2 = []
preds_mlp_2 = []
preds_ada_2 = []
preds_knn_2 = []
preds_lgbm_2 = []
preds_cat_2 = []
preds_xgb_2 = []

preds_lr_2 = []
preds_dt_2 = []

for i in range(0,len(preds_dnn_prob)):  
    # print(i)
    # print(preds_dnn_prob[i][y_test_reset_index.values[i][0]])
    preds_dnn_2.append(preds_dnn_prob[i][y_test_reset_index.values[i][0]])
    preds_svm_2.append(preds_svm_prob[i][y_test_reset_index.values[i][0]])
    preds_rf_2.append(preds_rf_prob[i][y_test_reset_index.values[i][0]])
    preds_mlp_2.append(preds_mlp_prob[i][y_test_reset_index.values[i][0]])
    preds_ada_2.append(preds_ada_prob[i][y_test_reset_index.values[i][0]])
    preds_knn_2.append(preds_knn_prob[i][y_test_reset_index.values[i][0]])
    preds_lgbm_2.append(preds_lgbm_prob[i][y_test_reset_index.values[i][0]])
    preds_cat_2.append(preds_cat_prob[i][y_test_reset_index.values[i][0]])
    preds_xgb_2.append(preds_xgb_prob[i][y_test_reset_index.values[i][0]])
    preds_lr_2.append(preds_lr_prob[i][y_test_reset_index.values[i][0]])
    preds_dt_2.append(preds_dt_prob[i][y_test_reset_index.values[i][0]])

    

In [ ]:
with open(output_file_name, "a") as f: print('------------------------------------------------------------------', file = f)
with open(output_file_name, "a") as f: print('------------------------------------------------------------------', file = f)
with open(output_file_name, "a") as f: print('------------------------------------------------------------------', file = f)

with open(output_file_name, "a") as f: print('------------START of STRONGER LEARNER - STACK 01 -----------------', file = f)


# Stack the vectors horizontally to create a matrix
column_features = ['dnn','rf','lgbm','ada','knn','mlp','svm','cat','xgb','lr','dt','label']
training_matrix2 = np.column_stack((
                          preds_dnn_2,
                          preds_rf_2,
                          preds_lgbm_2,
                          preds_ada_2,
                          preds_knn_2, 
                          preds_mlp_2,
                          preds_svm_2,
                          preds_cat_2,
                          preds_xgb_2,
                          preds_lr_2,
                          preds_dt_2,
                          y_test
                          ))

training_matrix = np.column_stack((
                          preds_dnn,
                          preds_rf,
                          preds_lgbm,
                          preds_ada,
                          preds_knn, 
                          preds_mlp,
                          preds_svm,
                          preds_cat,
                          preds_xgb,
                          preds_lr,
                          preds_dt,
                        #   preds
                          y_test
                          ))
# Print the resulting matrix
print(training_matrix)
print(training_matrix2)

[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 2. 2. ... 2. 1. 2.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.18495913 0.99566424 1.         ... 0.99535084 0.97001687 1.        ]
 [1.26331687 0.90148319 0.99954063 ... 0.99473733 0.97803335 0.        ]
 [2.15649462 0.95238536 0.99744784 ... 0.95716333 0.92120772 0.        ]
 ...
 [0.18928634 0.99566424 1.         ... 0.99535084 0.97168964 1.        ]
 [0.03016039 0.79127666 1.         ... 0.98627687 0.41109639 2.        ]
 [2.55233359 0.98269546 0.99944934 ... 0.99259925 0.97037767 0.        ]]


In [ ]:
df_level_00_0 = pd.DataFrame(training_matrix, columns=column_features)
df_level_00_1 = pd.DataFrame(training_matrix2, columns=column_features)

In [ ]:

# Assuming df is your DataFrame
if feature_selection_bit == 1:

    df_level_00_1.to_csv('base_models_prob_feature_selection.csv', index=False)
    df_level_00_0.to_csv('base_models_class_feature_selection.csv', index=False)
    
if feature_selection_bit == 0:

    df_level_00_1.to_csv('base_models_prob_all_features.csv', index=False)
    df_level_00_0.to_csv('base_models_class_all_features.csv', index=False)

In [ ]:
df_level_00_1


,dnn,rf,lgbm,ada,knn,mlp,svm,cat,xgb,lr,label
0,0.184959,0.995664,1.000000,0.269449,1.0,0.999999,0.091581,0.998595,0.995351,0.970017,1.0
1,1.263317,0.901483,0.999541,0.256679,1.0,1.000000,0.262305,0.997341,0.994737,0.978033,0.0
2,2.156495,0.952385,0.997448,0.227039,1.0,0.992756,0.016995,0.975448,0.957163,0.921208,0.0
3,2.361415,0.773010,0.908986,0.240320,1.0,0.993737,0.954099,0.888607,0.795008,0.965679,0.0
4,0.176913,0.995534,0.999906,0.270092,1.0,1.000000,0.022950,0.998703,0.995351,0.978102,1.0
...,...,...,...,...,...,...,...,...,...,...,...
44551,2.543204,0.982695,0.999375,0.269643,1.0,0.998290,0.968287,0.996241,0.993117,0.971517,0.0
44552,2.522557,0.991618,0.999552,0.269643,1.0,0.999442,0.674171,0.997375,0.993117,0.977973,0.0
44553,0.189286,0.995664,1.000000,0.269449,1.0,0.999999,0.027936,0.998696,0.995351,0.971690,1.0
44554,0.030160,0.791277,1.000000,0.265837,1.0,0.999998,0.001068,0.993283,0.986277,0.411096,2.0


In [ ]:
df_level_00_0

,dnn,rf,lgbm,ada,knn,mlp,svm,cat,xgb,lr,label
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
44551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44553,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
44554,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0


In [ ]:
# y_01 = df_level_01.pop('label')
# X_01 = df_level_01
# df_level_01 = df_level_01.assign(label = y_01)

In [ ]:
# X_01

In [ ]:
# y_01

In [ ]:
# df_level_01

In [ ]:
# split = 0.7

# X_train_01,X_test_01, y_train_01, y_test_01 = sklearn.model_selection.train_test_split(X_01, y_01, train_size=split)

In [ ]:
# from tabulate import tabulate

# # Assuming data is a 110x4 list, where each row is a sublist
# # data =  [["Row {} Col {}".format(i + 1, j + 1) for j in range(4)] for i in range(110)]
# data = [["" for _ in range(3)] for _ in range(12)]

# # Manually insert data at specific row and column
# # data[0][0] = "ADA"
# # data[1][0] = "DNN"
# # data[2][0] = "SVM"
# # data[3][0] = "ADA"
# # data[4][0] = "DNN"
# # data[2][0] = "SVM"


# names_models = ['ADA',
#                 'SVM',
#                 'DNN',
#                 'MLP',
#                 'KNN',
#                 'CAT',
#                 'XGB',
#                 'LGBM',
#                 'RF',
#                 'LR',
#                 'VOTING'
#                 ]
# level_00_f1 = [ada_f1_00,
#                 svm_f1_00,
#                 dnn_f1_00,
#                 mlp_f1_00,
#                 knn_f1_00,
#                 cat_f1_00,
#                 xgb_f1_00,
#                 lgbm_f1_00,
#                 rf_f1_00,
#                 lr_f1_00,
#                 voting_f1_00]  

                 

# for i in range(0,len(names_models)):
#     data[i][0] =  names_models[i]
#     data[i][1] = level_00_f1[i]
#     data[i][2] = level_01_f1[i]


 
# # data[0][1] = ada_acc_00
# # data

# # Define column headers
# headers = ["F1", "Level 00", "Level 01"]

# # Print the table
# table = tabulate(data, headers=headers, tablefmt="grid")
# print(table)
# with open(output_file_name, "a") as f: print(table, file = f)


In [ ]:
# lr_acc_00 = 0 
# voting_acc_00 = 0

# lr_pre_00 = 0 
# voting_pre_00 = 0

# lr_rec_00 = 0 
# voting_rec_00 = 0

# lr_f1_00 = 0 
# voting_f1_00 = 0

In [ ]:
from tabulate import tabulate

# Assuming data is a 110x4 list, where each row is a sublist
# data =  [["Row {} Col {}".format(i + 1, j + 1) for j in range(4)] for i in range(110)]
data = [["" for _ in range(5)] for _ in range(20)]

# Manually insert data at specific row and column
# data[0][0] = "ADA"
# data[1][0] = "DNN"
# data[2][0] = "SVM"
# data[3][0] = "ADA"
# data[4][0] = "DNN"
# data[2][0] = "SVM"


# names_models = ['ADA',
#                 'SVM',
#                 'DNN',
#                 'MLP',
#                 'KNN',
#                 'CAT',
#                 'XGB',
#                 'LGBM',
#                 'RF',
#                 'LR',
#                 'VOTING',
#                 '   '
#                 ]

# names_models = ['ADA',
#                 'SVM',
#                 'DNN',
#                 'MLP',
#                 'KNN',
#                 'CAT',
#                 'XGB',
#                 'LGBM',
#                 'RF',
#                 'LR',
#                 'DT',
#                 # 'VOTING',
#                 'Bag_svm',
#                 'Bag_knn',
#                 'Bag_DT',
#                 'Bag_LR',
#                 'Bag_mlp',
#                 # 'avg',
#                 # 'weighed_avg'
#                 ]

names_models = ['ADA',
                'SVM',
                'DNN',
                'MLP',
                'KNN',
                'CAT',
                'XGB',
                'LGBM',
                'RF',
                'LR',
                'DT',
                # 'VOTING',
                'Bag_svm',
                'Bag_knn',
                'Bag_DT',
                'Bag_LR',
                'Bag_mlp',

                'Bag_rf',
                'Bag_ada',
                'Bag_lgbm',
                # 'Bag_xgb',
                'Bag_cat',
                'Bag_comb',

                # 'avg',
                # 'weighed_avg'
                ]


level_00_acc = [ada_acc_00,
                svm_acc_00,
                dnn_acc_00,
                mlp_acc_00,
                knn_acc_00,
                cat_acc_00,
                xgb_acc_00,
                lgbm_acc_00,
                rf_acc_00,
                lr_acc_00,
                dt_acc_00,
                # voting_acc_00,
                bag_svm_acc_00,
                bag_knn_acc_00,
                bag_dt_acc_00,
                bag_lr_acc_00,
                bag_mlp_acc_00,
               
                bag_rf_acc_00,
                bag_ada_acc_00,
                bag_lgbm_acc_00,

                bag_cat_acc_00,
                bag_comb_acc_00,
               
               
                
                # avg_acc_00,
                # weighed_avg_acc_00
                ]  

                # ]  

level_00_pre = [ada_pre_00,
                svm_pre_00,
                dnn_pre_00,
                mlp_pre_00,
                knn_pre_00,
                cat_pre_00,
                xgb_pre_00,
                lgbm_pre_00,
                rf_pre_00,
                lr_pre_00,
                dt_pre_00,
                # voting_pre_00,
                bag_svm_pre_00,
                bag_knn_pre_00,
                bag_dt_pre_00,
                bag_lr_pre_00,
                bag_mlp_pre_00,

                bag_rf_pre_00,
                bag_ada_pre_00,
                bag_lgbm_pre_00,

                bag_cat_pre_00,
                bag_comb_pre_00,
               
                # avg_pre_00,
                # weighed_avg_pre_00
                ]  

level_00_rec = [ada_rec_00,
                svm_rec_00,
                dnn_rec_00,
                mlp_rec_00,
                knn_rec_00,
                cat_rec_00,
                xgb_rec_00,
                lgbm_rec_00,
                rf_rec_00,
                lr_rec_00,
                dt_rec_00,
                # voting_rec_00,
                bag_svm_rec_00,
                bag_knn_rec_00,
                bag_dt_rec_00,
                bag_lr_rec_00,
                bag_mlp_rec_00,

                bag_rf_rec_00,
                bag_ada_rec_00,
                bag_lgbm_rec_00,

                bag_cat_rec_00,
                bag_comb_rec_00,
               
                # avg_rec_00,
                # weighed_avg_rec_00
                ]  

level_00_f1 = [ada_f1_00,
                svm_f1_00,
                dnn_f1_00,
                mlp_f1_00,
                knn_f1_00,
                cat_f1_00,
                xgb_f1_00,
                lgbm_f1_00,
                rf_f1_00,
                lr_f1_00,
                dt_rec_00,
                # voting_f1_00,
                bag_svm_f1_00,
                bag_knn_f1_00,
                bag_dt_f1_00,
                bag_lr_f1_00,
                bag_mlp_f1_00,

                bag_rf_f1_00,
                bag_ada_f1_00,
                bag_lgbm_f1_00,

                bag_cat_f1_00,
                bag_comb_f1_00,
               
                # avg_f1_00,
                # weighed_avg_f1_00
                ]                   

for i in range(0,len(names_models)):
    data[i][0] =  names_models[i]

    data[i][1] = level_00_acc[i]
    # data[i][2] = level_01_acc[i]

    data[i][2] = level_00_pre[i] 
    # data[i][4] = level_01_pre[i]

    data[i][3] = level_00_rec[i] 
    # data[i][6] = level_01_rec[i]

    data[i][4] = level_00_f1[i]
    # data[i][8] = level_01_f1[i]




 
# data[0][1] = ada_acc_00
# data

# Define column headers
# headers = ["Models", "ACC-00", " ACC-01","PRE-00", " PRE-01","REC-00", " REC-01","F1-00", " F1-01",]
headers = ["Models", "ACC-00","PRE-00","REC-00","F1-00"]


# Print the table
table = tabulate(data, headers=headers, tablefmt="grid")
print(table)
with open(output_file_name, "a") as f: print(table, file = f)


+----------+----------+----------+----------+----------+
| Models   |   ACC-00 |   PRE-00 |   REC-00 |    F1-00 |
+==========+==========+==========+==========+==========+
| ADA      | 0.757474 | 0.49015  | 0.518571 | 0.495145 |
+----------+----------+----------+----------+----------+
| SVM      | 0.866909 | 0.518517 | 0.488025 | 0.486543 |
+----------+----------+----------+----------+----------+
| DNN      | 0.839662 | 0.336971 | 0.380104 | 0.356995 |
+----------+----------+----------+----------+----------+
| MLP      | 0.966626 | 0.726111 | 0.743367 | 0.734398 |
+----------+----------+----------+----------+----------+
| KNN      | 0.976412 | 0.847044 | 0.765066 | 0.764238 |
+----------+----------+----------+----------+----------+
| CAT      | 0.968601 | 0.735253 | 0.734851 | 0.735038 |
+----------+----------+----------+----------+----------+
| XGB      | 0.95206  | 0.714165 | 0.689462 | 0.700174 |
+----------+----------+----------+----------+----------+
| LGBM     | 0.975514 | 0.75897

In [ ]:
# Combine data into a list of tuples for sorting
model_data = list(zip(names_models, level_00_acc, level_00_pre, level_00_rec, level_00_f1))

# Sort by F1-00 score in descending order
model_data_sorted = sorted(model_data, key=lambda x: x[4], reverse=True)

# Separate the sorted data back into individual lists
sorted_names_models, sorted_level_00_acc, sorted_level_00_pre, sorted_level_00_rec, sorted_level_00_f1 = zip(*model_data_sorted)

# Assign the sorted data to the table
for i in range(len(sorted_names_models)):
    data[i][0] = sorted_names_models[i]
    data[i][1] = sorted_level_00_acc[i]
    data[i][2] = sorted_level_00_pre[i] 
    data[i][3] = sorted_level_00_rec[i] 
    data[i][4] = sorted_level_00_f1[i]

# Define column headers
headers = ["Models", "ACC-00", "PRE-00", "REC-00", "F1-00"]

# Print the table
table = tabulate(data, headers=headers, tablefmt="grid")
with open(output_file_name, "a") as f: print('Summary table - LEVEL 00', file = f)

if feature_selection_bit == 1: 
    with open(output_file_name, "a") as f: print('Feature Selection was applied', file = f)
else:
    with open(output_file_name, "a") as f: print('All features were used', file = f)


    
print(table)
with open(output_file_name, "a") as f: print(table, file = f)

+----------+----------+----------+----------+----------+
| Models   |   ACC-00 |   PRE-00 |   REC-00 |    F1-00 |
+==========+==========+==========+==========+==========+
| Bag_DT   | 0.979352 | 0.81605  | 0.789263 | 0.798081 |
+----------+----------+----------+----------+----------+
| Bag_lgbm | 0.980811 | 0.790636 | 0.774846 | 0.777762 |
+----------+----------+----------+----------+----------+
| LGBM     | 0.975514 | 0.758973 | 0.783135 | 0.767766 |
+----------+----------+----------+----------+----------+
| KNN      | 0.976412 | 0.847044 | 0.765066 | 0.764238 |
+----------+----------+----------+----------+----------+
| Bag_knn  | 0.976546 | 0.779105 | 0.765777 | 0.762737 |
+----------+----------+----------+----------+----------+
| Bag_cat  | 0.969118 | 0.737696 | 0.736019 | 0.736815 |
+----------+----------+----------+----------+----------+
| Bag_mlp  | 0.969005 | 0.73386  | 0.739233 | 0.736423 |
+----------+----------+----------+----------+----------+
| CAT      | 0.968601 | 0.73525

In [ ]:
# implement time table
from tabulate import tabulate

names_models = ['ADA',
                'SVM',
                'DNN',
                'MLP',
                'KNN',
                'CAT',
                'XGB',
                'LGBM',
                'RF',
                'LR',
                'DT',
                # 'VOTING',
                'Bag_svm',
                'Bag_knn',
                'Bag_DT',
                'Bag_LR',
                'Bag_mlp',

                'Bag_rf',
                'Bag_ada',
                'Bag_lgbm',
                # 'Bag_xgb',
                'Bag_cat',
                'Bag_comb',
                # 'avg',
                # 'weighed_avg'
                ]

data = [["" for _ in range(2)] for _ in range(len(names_models))]

level_00_time = [
                ada_time_00,
                svm_time_00,
                dnn_time_00,
                mlp_time_00,
                knn_time_00,
                cat_time_00,
                xgb_time_00,
                lgbm_time_00,
                rf_time_00,
                lr_time_00,
                dt_time_00,
                # voting_time_00,
                bag_svm_time_00,
                bag_knn_time_00,
                bag_dt_time_00,
                bag_lr_time_00,
                bag_mlp_time_00,

                bag_rf_time_00,
                bag_ada_time_00,
                bag_lgbm_time_00,
                # bag_xgb_time_00,
                bag_cat_time_00,
                bag_comb_time_00,

                # avg_time_00,
                # weighed_avg_time_00
                ]  


# Combine data into a list of tuples for sorting
model_data = list(zip(names_models, level_00_time))

# Sort by F1-00 score in descending order
model_data_sorted = sorted(model_data, key=lambda x: x[1], reverse=False)

# Separate the sorted data back into individual lists
sorted_names_models, sorted_level_00_time = zip(*model_data_sorted)

# Assign the sorted data to the table
for i in range(len(sorted_names_models)):
    data[i][0] = sorted_names_models[i]
    data[i][1] = sorted_level_00_time[i]

# Define column headers
headers = ["Models", "time-00(sec)"]


# Print the table
table = tabulate(data, headers=headers, tablefmt="grid")
with open(output_file_name, "a") as f: print('Time is counted is seconds', file = f)
print(table)
with open(output_file_name, "a") as f: print(table, file = f)
end_program = time.time()
time_program = end_program - start_program
with open(output_file_name, "a") as f: print('Running time of entire program is:', time_program ,' seconds',file = f)

+----------+--------------------+
| Models   | time-00(sec)       |
+==========+====================+
| RF       | 0.334575891494751  |
+----------+--------------------+
| LGBM     | 1.205275058746338  |
+----------+--------------------+
| Bag_knn  | 1.236325740814209  |
+----------+--------------------+
| ADA      | 2.2060389518737793 |
+----------+--------------------+
| CAT      | 2.634706974029541  |
+----------+--------------------+
| LR       | 3.022087335586548  |
+----------+--------------------+
| KNN      | 3.2511746883392334 |
+----------+--------------------+
| SVM      | 3.336155652999878  |
+----------+--------------------+
| Bag_mlp  | 4.391276597976685  |
+----------+--------------------+
| VOTING   | 5.176404714584351  |
+----------+--------------------+
| Bag_ada  | 10.407995462417603 |
+----------+--------------------+
| XGB      | 10.638646364212036 |
+----------+--------------------+
| Bag_svm  | 16.19873547554016  |
+----------+--------------------+
| Bag_rf   | 1